In [4]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import pipecaster as pc

Xs, y, _ = pc.make_multi_input_classification(n_informative_Xs=3,
                                              n_random_Xs=7)
clf = pc.MultichannelPipeline(n_channels=10)
clf.add_layer(GradientBoostingClassifier(), pipe_processes='max')
clf.add_layer(pc.MultichannelPredictor(SVC()))
pc.cross_val_score(clf, Xs, y, cv=3)

[0.8529411764705883, 0.9099264705882353, 0.8511029411764706]

In [2]:
clf.fit(X, y)
print('Models selected by the SelectiveStack:')
[p for i, p in enumerate(predictors) if i in clf.get_support()]

Models selected by the SelectiveStack:


[GradientBoostingClassifier(), RandomForestClassifier()]